In [1]:
import pandas as pd
import numpy as np

In [297]:
df8 = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - №4 ШЖОББМ.xlsx", sheet_name='data')

In [298]:
for index, subject in enumerate(df8['subject']):
    if not '4' in subject:
        df8.loc[index, 'subject'] = '8' + df8.loc[index, 'subject']

In [299]:
file_path = r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - №4 ШЖОББМ.xlsx"
# df8.to_excel(file_path, index=False, sheet_name='data')

---

In [300]:
def fio_change(df):
    df['fio'] = df['fio'].apply(lambda x: ' '.join(str(x).split()[:2]) if len(str(x).split()) > 2 else x)
    return df

In [301]:
db = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\63 сш.xlsx").map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)
db = fio_change(db)
db = db[['fio', 'lang']]

In [311]:
file_paths = {
    'df1' : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Аққулы СОР и СОЧ.xlsx",
    'df2' : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Баянаул, З.Акишев СОР и СОЧ.xlsx",
    'df3' : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Тереңкөл СОР, СОЧ-1.xlsx"
}

subject_map = {
    'английский язык' : 'ENFL',
    'ағылшын тілі' : 'ENFL',
    'биология' : 'BIOL',
    'география' : 'GEOG',
    'жаратылыстану' : 'SCIE',
    'казахский язык' : 'KZFL',
    'математика' : 'MATH',
    'русский язык' : 'RUFL',
    'русский язык и литература' : 'RUFL',
    'физика' : 'PHYS',
    'химия' : 'CHEM',
    'қазақ тілі' : 'KZFL'
}

dfs = {}
for key, path in file_paths.items():
    dfs[key] = pd.read_excel(path, sheet_name='data')
    dfs[key] = dfs[key].map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)
    dfs[key]['subject'] = dfs[key]['subject'].str.lower()
    dfs[key]['subject'] = dfs[key]['subject'].map(subject_map)
    dfs[key] = fio_change(dfs[key])

In [312]:
# dfs['df3']

In [313]:
merged_dict = {}
noLang_names = set()
for key, df in dfs.items():
    merged = pd.merge(df, db, on='fio', how='left')
    merged['fio'] = merged['fio'].ffill()

    for index, lang in enumerate(merged['lang']):
        if (lang == 1) and (merged.loc[index, 'subject'] == 'RUFL'):
            merged.loc[index, 'subject'] = 'RUSL'
        elif (lang == 2) and (merged.loc[index, 'subject'] == 'KZFL'):
            merged.loc[index, 'subject'] = 'KZSL'
    
    merged = merged.drop_duplicates()

    for fio in merged['fio'].unique():
        table = merged[merged['fio'] == fio]
        table.loc[:, 'lang'] = table['lang'].ffill()
        merged.loc[merged['fio'] == fio, 'lang'] = table['lang'].unique()[0]
    
    naLangFio = merged[merged['lang'].isna()]['fio'].unique()
    noLang_names.update(name for name in naLangFio)
    
    merged_dict[key] = merged

In [314]:
noLang_names

{'Абдусат Әлихан',
 'Амангелді Батырхан',
 'Анас Сырғалым',
 'Арын Әнел',
 'Асқаров Мансұр',
 'Болатов Мақсат',
 'Десесін Ерасыл',
 'Жетписбаев Ансар',
 'Иманбаев Саян',
 'Каирбеков Айан',
 'Каиргельдинов Алнурат',
 'Канат Алина',
 'Каржасбаев Ерасыл',
 'Марат Алихан',
 'Мукенова Адема',
 'Мухамеджан Мухамедияр',
 'Мұқан Надия',
 'Нугуманов Арман',
 'Садыгулов Расул',
 'Сағымбай Мейіржан',
 'Төлеутай Ғабит',
 'Төлеухан Гауһар',
 'Харахат Айтолкын',
 'Қаиырғали Айым',
 'Қалиқайдар Гауһар',
 'Қалтай Аяулым',
 'Қалымбек Ернұр'}

In [315]:
for _, df in merged_dict.items():
    display(df)

,fio,subject,grade,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max,lang
0,Бақытбек Бинур,BIOL,8,15,25,13.0,14.0,9.0,14.0,23,30,1.0
7,Бақытбек Бинур,MATH,8,12,13,NaN,NaN,NaN,NaN,18,20,1.0
14,Бақытбек Бинур,PHYS,8,12,13,12.0,13.0,NaN,NaN,22,25,1.0
21,Бақытбек Бинур,GEOG,8,9,10,8.0,9.0,10.0,12.0,22,25,1.0
28,Бақытбек Бинур,RUSL,8,9,9,13.0,13.0,NaN,NaN,30,30,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1237,Юлдашев Бахтжан,BIOL,8,15,19,7.0,11.0,15.0,19.0,19,30,2.0
1244,Юлдашев Бахтжан,MATH,8,10,15,NaN,NaN,NaN,NaN,12,20,2.0
1251,Юлдашев Бахтжан,PHYS,8,8,12,9.0,13.0,NaN,NaN,13,25,2.0
1258,Юлдашев Бахтжан,GEOG,8,4,8,5.0,9.0,8.0,12.0,19,25,2.0


,fio,subject,grade,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max,lang
0,Аужан Ерназар,KZFL,8,4,7,3.0,7.0,NaN,NaN,18,30,1.0
2,Аужан Ерназар,RUSL,8,7,10,12.0,17.0,NaN,NaN,19,30,1.0
4,Аужан Ерназар,ENFL,8,7,10,7.0,10.0,NaN,NaN,18,24,1.0
6,Аужан Ерназар,CHEM,8,8,14,8.0,15.0,9.0,15.0,12,25,1.0
8,Аужан Ерназар,BIOL,8,10,17,7.0,12.0,9.0,15.0,15,30,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1284,Серік Алшын,KZFL,4,5,10,6.0,10.0,NaN,10.0,8,18,1.0
1287,Серік Алшын,SCIE,4,6,10,6.0,10.0,NaN,10.0,8,18,1.0
1290,Талғат Медина,MATH,4,9,10,8.0,10.0,9.0,10.0,15,18,1.0
1293,Талғат Медина,KZFL,4,9,10,9.0,10.0,NaN,10.0,15,18,1.0


,fio,subject,grade,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max,lang
0,Айжанова Аружан,BIOL,8,11.0,25.0,6.0,NaN,10.0,NaN,19.0,25.0,1.0
6,Айжанова Аружан,MATH,8,9.0,13.0,NaN,NaN,NaN,NaN,12.0,20.0,1.0
7,Айжанова Аружан,PHYS,8,7.0,10.0,7.0,10.0,NaN,NaN,16.0,25.0,1.0
8,Айжанова Аружан,GEOG,8,8.0,10.0,7.0,9.0,NaN,NaN,16.0,25.0,1.0
9,Айжанова Аружан,CHEM,8,8.0,13.0,9.0,15.0,12.0,15.0,19.0,25.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
488,Сейтахметова Аделия,MATH,4,8.0,9.0,8.0,8.0,10.0,11.0,16.0,18.0,1.0
489,Сейтахметова Аделия,SCIE,4,7.0,8.0,8.0,10.0,NaN,NaN,16.0,18.0,1.0
490,Сулейменов Рамазан,KZFL,4,5.0,7.0,5.0,7.0,NaN,NaN,8.0,18.0,1.0
493,Сулейменов Рамазан,MATH,4,5.0,9.0,5.0,8.0,5.0,11.0,8.0,18.0,1.0


In [307]:
dfs['df1'].columns

Index(['fio', 'subject', 'grade', 'sor1', 'sor1_max', 'sor2', 'sor2_max',
       'sor3', 'sor3_max', 'soch', 'soch_max'],
      dtype='object')

In [308]:
subject_set = set()
for key, df in dfs.items():
    subjects = df['subject'].unique()
    for subject in subjects:
        subject_set.add(subject)

In [309]:
df8

,fio,subject,sor1,sor2,sor3,soch
0,Алмаханова Гаухар Қанатқызы,88KZFL,6\3,7\3,NaN,30\14
1,Алмаханова Гаухар Қанатқызы,88KZSL,7\3,8\3,NaN,25\15
2,Алмаханова Гаухар Қанатқызы,88ENFL,10\5,10\6,NaN,24\14
3,Алмаханова Гаухар Қанатқызы,88MATH,14\9,NaN,NaN,20\15
4,Алмаханова Гаухар Қанатқызы,88CHEM,14\14,15\11,15\11,24\15
...,...,...,...,...,...,...
643,Төлеуғали Хасан Жасұланұлы,4SCIE,10\8,10\7,NaN,18\13
644,Хобдабай Нұрбақыт Гайдарқызы,4MATH,10\5,10\4,10\5,18\8
645,Хобдабай Нұрбақыт Гайдарқызы,4KZFL,10\5,10\5,NaN,18\13
646,Хобдабай Нұрбақыт Гайдарқызы,4RUFL,7\3,9\3,NaN,18\7


In [310]:
dfs

{'df1':                  fio subject  grade  sor1  sor1_max  sor2  sor2_max  sor3  \
 0     Бақытбек Бинур    BIOL      8    15        25  13.0      14.0   9.0   
 1     Бақытбек Бинур    MATH      8    12        13   NaN       NaN   NaN   
 2     Бақытбек Бинур    PHYS      8    12        13  12.0      13.0   NaN   
 3     Бақытбек Бинур    GEOG      8     9        10   8.0       9.0  10.0   
 4     Бақытбек Бинур    RUFL      8     9         9  13.0      13.0   NaN   
 ..               ...     ...    ...   ...       ...   ...       ...   ...   
 200  Юлдашев Бахтжан    BIOL      8    15        19   7.0      11.0  15.0   
 201  Юлдашев Бахтжан    MATH      8    10        15   NaN       NaN   NaN   
 202  Юлдашев Бахтжан    PHYS      8     8        12   9.0      13.0   NaN   
 203  Юлдашев Бахтжан    GEOG      8     4         8   5.0       9.0   8.0   
 204  Юлдашев Бахтжан    RUFL      8     8        10   8.0       9.0   NaN   
 
      sor3_max  soch  soch_max  
 0        14.0    23  

---

In [12]:
df61 = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Карагандинская область\COPY - СОР и СОЧ 15.xlsx", sheet_name='data').map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

In [16]:
df61[df61['subject'].isin(['жаратылыстану', 'дүниетану'])]

,fio,grade,subject,sor1,sor2,sor3,soch
1214,Артықбаев Санжар,4,жаратылыстану,10/8,10/7,NaN,18/13
1215,Артықбаев Санжар,4,дүниетану,10/8,NaN,NaN,NaN
1221,Дюсембекова Каусар,4,жаратылыстану,10/7,10/7,NaN,18/14
1222,Дюсембекова Каусар,4,дүниетану,10/7,NaN,NaN,NaN
1228,Жарылғасын Жалғас,4,жаратылыстану,10/7,10/5,NaN,18/14
...,...,...,...,...,...,...,...
2006,Тұрсынбек Рахымжан,4,дүниетану,10/7,NaN,NaN,NaN
2012,Утегенова Адина,4,жаратылыстану,10/10,10/9,NaN,18/8
2013,Утегенова Адина,4,дүниетану,10/10,NaN,NaN,NaN
2019,Шаймаханбет Амина,4,жаратылыстану,10/10,10/10,NaN,18/10


In [ ]:
subject_map = {
    'қазақ тілі' : 'KZFL',
    'қазақ әдебиеті' : 'KZFL', 
    'орыс тілі мен әдебиеті' : 'RUFL',
    'шетел тілі' : 'ENFL', 
    'алгебра' : 'MATH', 
    'геометрия' : 'MATH', 
    'география' : 'GEOG', 
    'биология' : 'BIOL',
    'химия' : 'CHEM',
    'физика' : 'PHYS', 
    'Қазақстан тарихы' : 'HOKZ', 
    'дүниежүзі тарихы' : 'HOTW',
    'информатика' : 'CSCI', 
    'әдебиеттік оқу' : , 'орыс тілі', 'математика',
    'жаратылыстану', 'дүниетану'
}